In [1]:
import sys
sys.path.append("../../src")
import torch
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn.functional as F

import glob
import os
from datetime import datetime
import time
import math
from tqdm import tqdm

from itertools import repeat
from torch.nn.parameter import Parameter
import collections
import matplotlib
from torch_utils import *
from ExplicitModels import *
from visualization import *
# matplotlib.use('Agg')

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [3]:
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor(), 
                                            torchvision.transforms.Normalize(mean=(0.0,), std=(1.0,))])

mnist_dset_train = torchvision.datasets.MNIST('../../data', train=True, transform=transform, target_transform=None, download=True)
train_loader = torch.utils.data.DataLoader(mnist_dset_train, batch_size=20, shuffle=True, num_workers=0)

mnist_dset_test = torchvision.datasets.MNIST('../../data', train=False, transform=transform, target_transform=None, download=True)
test_loader = torch.utils.data.DataLoader(mnist_dset_test, batch_size=20, shuffle=False, num_workers=0)

In [4]:
# activation = torch.sigmoid
activation = F.relu
architecture = [784, 128, 64, 10]
lambda_ = 0.9999
epsilon = 0.1
neural_lr_start = 0.1
neural_lr_stop = 0.05
neural_lr_rule = "constant"
neural_lr_decay_multiplier = 0.005
neural_dynamic_iterations = 50

model = CorInfoMax_wAutoGrad(architecture, lambda_, epsilon, activation, optimizer_type = "sgd",
                             optim_lr = 0.05, use_stepLR = True, stepLR_step_size = 10*3000)

In [5]:
# optim_params = []
# for idx in range(len(model.Wff)):
#     for key_ in ["weight", "bias"]:
#         optim_params.append(  {'params': model.Wff[idx][key_], 'lr': lr_start["ff"]}  )

In [6]:
# optimizer = torch.optim.Adam(optim_params, maximize = True)

In [ ]:
trn_acc_list = []
tst_acc_list = []

n_epochs = 50

for epoch_ in range(n_epochs):
    if epoch_ > 12:
        neural_lr_start = 0.05
    if epoch_ > 17:
        neural_lr_start = 0.03
    for idx, (x, y) in tqdm(enumerate(train_loader)):
        x, y = x.to(device), y.to(device)
        #x = activation_inverse(x.view(x.size(0),-1).T, "sigmoid")
        x = x.view(x.size(0),-1).T
        y_one_hot = F.one_hot(y, 10).to(device).T
        #y_one_hot = 0.94 * y_one_hot + 0.03 * torch.ones(*y_one_hot.shape, device = device)
        
        model.batch_step(  x, y_one_hot, neural_lr_start, neural_lr_stop, neural_lr_rule,
                           neural_lr_decay_multiplier, neural_dynamic_iterations,
                         )

    trn_acc = evaluatePC(model, train_loader, device, False, 
                         printing = False)
    tst_acc = evaluatePC(model, test_loader, device, False, 
                         printing = False)
    trn_acc_list.append(trn_acc)
    tst_acc_list.append(tst_acc)
    
    print("Epoch : {}, Train Accuracy : {}, Test Accuracy : {}".format(epoch_+1, trn_acc, tst_acc))

3000it [05:38,  8.85it/s]
1it [00:00,  6.95it/s]

Epoch : 1, Train Accuracy : 0.9265666666666666, Test Accuracy : 0.9235


3000it [05:40,  8.80it/s]
1it [00:00,  7.71it/s]

Epoch : 2, Train Accuracy : 0.9401833333333334, Test Accuracy : 0.9352


3000it [06:23,  7.82it/s]
1it [00:00,  6.92it/s]

...

3000it [07:17,  6.85it/s]
1it [00:00,  6.98it/s]

Epoch : 4, Train Accuracy : 0.9481833333333334, Test Accuracy : 0.9423


3000it [07:00,  7.13it/s]
1it [00:00,  7.00it/s]

Epoch : 5, Train Accuracy : 0.9503166666666667, Test Accuracy : 0.9437


235it [00:32,  7.51it/s]